<a href="https://colab.research.google.com/github/likeJoseph/helloworld/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!pip install mysql-connector-python



---



In [153]:
import mysql.connector
import sys
import pandas as pd
import matplotlib.pyplot as plt
# import mysql

**database 연결 정보 입력**

In [192]:
# Connect to the database
HOST = "147.46.15.238" # database 를 생성/연결할 서버 IP
PORT = "7000"
USER = "DBilgyo_chong"
PASSWD = "DBilgyo_chong"
DB = "DBilgyo_chong"

# mysql 을 서버에서 활용할 수 있도록 연결해준다.
connection = mysql.connector.connect(
    host=HOST,
    port=PORT,
    user=USER,
    passwd=PASSWD,
    db=DB,
    autocommit=True  # to create table permanently
)

# Cursor 를 통해 서버의 database 와 커뮤니케이트 한다.
cur = connection.cursor(dictionary=True)

# SQL query 를 받아 해당 query를 보내고 그 결과 값을 dataframe으로 저장해 return 해주는 함수
def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

In [194]:
# ratings, item_similarity, user_similarity
# test connection
query = "SHOW tables;"     # 서버의 database 에 보내고 싶은 sql query
cur.execute(query)         # cur.execute() 을 통해 query 를 서버로 송출
a = cur.fetchall()         # cur.fetchall() 을 통해 query 실행 후 결과값 반환
print(pd.DataFrame(a))

   Tables_in_DBilgyo_chong
0                 customer
1              geolocation
2          item_similarity
3               order_info
4               order_item
5            order_payment
6             order_review
7                  product
8                  ratings
9                   seller
10         user_similarity


## <b> 1. Popularity-based Recommendation (Popularity by rating count)

In [157]:
query = '''
SELECT * FROM ratings
'''
df = get_output(query)

In [224]:
# 1번 답
query = '''
SELECT item, COUNT(user) AS count
FROM ratings
WHERE rating IS NOT NULL
GROUP BY item
ORDER BY count DESC
'''
get_output(query)


,item,count
0,28,226
1,22,212
2,54,201
3,59,194
4,203,194
...,...,...
448,5,32
449,452,32
450,278,31
451,435,31




---

## <B> 2. Popularity by average rating


In [222]:
# 답2
query = '''
SELECT item, ROUND(AVG(P),4) AS prediction
FROM (SELECT user, item, (rating-min)/(max-min) AS P
      FROM ratings LEFT JOIN (SELECT user, MAX(rating) AS max, MIN(rating) AS min FROM ratings GROUP BY user) R USING(user)) R2
GROUP BY item
ORDER BY prediction DESC
'''
get_output(query)

,item,prediction
0,316,0.8666
1,203,0.8634
2,439,0.8528
3,69,0.8458
4,4,0.8427
...,...,...
448,451,0.3215
449,450,0.3206
450,58,0.3096
451,63,0.2929




---

## <B> 3. IBCF

(1) 과거 데이터 불러오기

In [266]:
query = '''
SELECT *
FROM item_similarity
ORDER BY item_1
'''
get_output(query).head(10)

,item_1,item_2,sim
0,0,0,0.0
1,0,1,0.6
2,0,2,0.7
3,0,3,0.8
4,0,4,0.7
5,0,5,0.6
6,0,6,0.7
7,0,7,0.8
8,0,8,0.8
9,0,9,0.8


(2) 아이템마다 가장 유사도가 높은 이웃 K개를 구한다.

In [267]:
query = '''
SELECT *
FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC) AS ranking
FROM item_similarity) R
WHERE ranking <= 3
'''
get_output(query)

,item_1,item_2,sim,ranking
0,0,3,0.8,1
1,0,7,0.8,2
2,0,8,0.8,3
3,1,19,0.8,1
4,1,29,0.8,2
...,...,...,...,...
1354,451,21,0.7,2
1355,451,24,0.7,3
1356,452,1,0.7,1
1357,452,5,0.7,2


(3-1) 각 아이템의 유사도 합 구하기

In [257]:
query = '''
SELECT item_1, sum(sim) AS SUM
FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC) AS ranking
FROM item_similarity) R
WHERE ranking <= 3
GROUP BY item_1
'''
get_output(query)

,item_1,SUM
0,0,2.4
1,1,2.4
2,2,2.4
3,3,2.6
4,4,2.5
...,...,...
448,448,2.4
449,449,2.2
450,450,2.4
451,451,2.1


(3-2) 각 아이템마다 유사도를 유사도의 합으로 나누어 준다. (소수점 넷째자리까지)

In [277]:
query = '''
SELECT *, ROUND(sim/SUM,4) AS norm_sim
FROM (SELECT item_1, item_2, sim, ranking
            FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC) AS ranking FROM item_similarity) R
            WHERE ranking <= 3) A
      LEFT OUTER JOIN
      (SELECT item_1, sum(sim) AS SUM
            FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY item_1 ORDER BY sim DESC) AS ranking FROM item_similarity) R
            WHERE ranking <= 3
            GROUP BY item_1) B
      USING (item_1)
'''
get_output(query)

,item_1,item_2,sim,ranking,SUM,norm_sim
0,32,0,0.7,3,2.4,0.2917
1,19,2,0.7,3,2.3,0.3043
2,195,3,0.7,3,2.3,0.3043
3,205,1,0.7,3,2.3,0.3043
4,207,4,0.7,2,2.3,0.3043
...,...,...,...,...,...,...
1354,376,362,0.8,1,2.2,0.3636
1355,438,111,0.8,1,2.2,0.3636
1356,449,97,0.8,1,2.2,0.3636
1357,32,277,0.9,1,2.4,0.3750


(4) 아이템-사용도 평점 행렬에서 아이템의 평균으로 빈 공간 채우기

In [281]:
# 아이템의 평균 평점
query = '''
SELECT item, AVG(rating)
FROM ratings
GROUP BY item
'''
get_output(query)

,item,AVG(rating)
0,0,3.364130
1,1,3.990566
2,2,3.526316
3,3,4.036913
4,4,4.379310
...,...,...
448,448,3.215686
449,449,3.656250
450,450,2.571429
451,451,2.467742
